In [1]:
import numpy as np
import rasterio
from rasterio.features import shapes
from scipy.ndimage import gaussian_gradient_magnitude
import matplotlib.pyplot as plt

In [2]:
# Step 1: Load and Preprocess Magnetic Anomaly Data
def load_magnetic_data(file_path):
    """
    Load the magnetic anomaly data from a GeoTIFF file.
    Returns the dataset and the first data band.
    """
    # Open the GeoTIFF file
    dataset = rasterio.open(file_path)
    data = dataset.read(1)  # Read the first band
    return dataset, data

# Load the data from the specified path
file_path = '/Users/haritshah/Downloads/Mag Anom UTM18N NAD83.tif'
dataset, magnetic_data = load_magnetic_data(file_path)

In [3]:
# Step 2: Compute Magnetic Gradient
def compute_magnetic_gradient(data, sigma=1):
    """
    Compute the gradient magnitude of the magnetic anomaly data.
    """
    gradient = gaussian_gradient_magnitude(data, sigma=sigma)
    return gradient

In [6]:
# Step 3: Visualize Gradient (Fault Zone Potential) as Raster
def visualize_gradient_as_raster(gradient_data, dataset):
    """
    Visualize the gradient data (potential fault zones) as a raster overlay.
    """
    plt.figure(figsize=(12, 10))
    plt.imshow(
        gradient_data,
        cmap="hot",
        extent=(
            dataset.bounds.left,
            dataset.bounds.right,
            dataset.bounds.bottom,
            dataset.bounds.top,
        ),
    )
    plt.colorbar(label="Gradient Magnitude (Fault Zone Potential)")
    plt.title("Potential Fault Zones (Raster Overlay)")
    plt.xlabel("Easting (m)")
    plt.ylabel("Northing (m)")
    plt.grid(visible=True, color="gray", alpha=0.5)
    plt.show()

In [13]:
# Step 4: Export Results as GeoTIFF with 3 Bands (RGB)
def save_gradient_as_rgb_geotiff(gradient_data, reference_dataset, output_path, brightness_factor=1.5):
    """
    Save gradient data as a GeoTIFF file with 3 bands (RGB) using the metadata from the reference dataset.
    Adjust the brightness of the image using the brightness_factor.
    """
    # Normalize the gradient data to the range [0, 1]
    normalized_gradient = (gradient_data - np.min(gradient_data)) / np.ptp(gradient_data)
    normalized_gradient = np.clip(normalized_gradient * brightness_factor, 0, 1)  # Apply brightness factor and clip

    # Apply a custom colormap to emphasize red tones
    colormap = plt.get_cmap('Reds')
    rgb_image = colormap(normalized_gradient)[:, :, :3]  # Get RGB channels

    # Convert RGB image to uint8
    rgb_image = (rgb_image * 255).astype(np.uint8)

    # Get metadata from the reference dataset
    meta = reference_dataset.meta.copy()
    meta.update({
        'dtype': 'uint8',
        'count': 3,  # 3 bands for RGB
        'compress': 'lzw'
    })

    # Write the RGB image to a new GeoTIFF
    with rasterio.open(output_path, 'w', **meta) as dest:
        for i in range(3):
            dest.write(rgb_image[:, :, i], i + 1)

# Save gradient as GeoTIFF with 3 bands (RGB) with increased brightness
save_gradient_as_rgb_geotiff(magnetic_gradient, dataset, output_gradient_path, brightness_factor=1.5)

print(f"Gradient map saved as: {output_gradient_path}")


Gradient map saved as: /Users/haritshah/Desktop/Magnetic_Gradient_RGB.tif
